## Set Up

In [ ]:
import requests as re
import pandas as pd
import numpy as np

# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(0, '../..')

from APIKeys import unsplash_access_key

In [ ]:
#Basic api call function
def API(root, search_term, param, header=None):
    """
    'search_term' must be a string of valid end point queries 
            - as specified by the relevent documentation
            
    OUTPUT: JSON of results
   """
 
    #define endpoint
    if search_term is not None:
        #if search_term is defined add it to the root to make the endpoint
        endpoint = root + search_term
    else:
        #otherwise endpoint is just the root
        endpoint = root
    
    #GET
    response = re.get(endpoint, params = param, headers=header)
   
    #return status code and results
    status_code, results = response.status_code, response.json()
    
    #Let's make sure it worked
    if status_code != 200:
        print('Something went wrong!')
        print(status_code)
        
    return results

## Test

In [ ]:
params = {'client_id': unsplash_access_key}
unsplash_root = 'https://api.unsplash.com/'

test = API(unsplash_root, 'photos/random', param = params)

In [ ]:
response = requests.get(test['urls']['raw'])
img = Image.open(BytesIO(response.content)) #fairly slow, should limit how much im doing this
plt.imshow(img)

## Search by topic

In [ ]:
def unsplash_term_queries(term, pages = 1, orientation = 'landscape'):
    
    queries = []
    
    for i in range(0, pages):
        params = {'client_id': unsplash_access_key,
                  'query': term,
                  'page': i + 1,
                  'per_page': 30,
                  'orientation': orientation}
    
        queries.append(API(unsplash_root, 'search/photos', param = params)['results'])
        
    return queries

In [ ]:
def json_to_images(json_results):
    image_arrays = []
    
    for photo_data in json_results:
        response = requests.get(photo_data['urls']['raw'])
        img = Image.open(BytesIO(response.content))
    
        image_arrays.append(np.asarray(img))
        
    return image_arrays

def queries_to_images(queries):
    
    images = []
    
    for query in queries:
        temp = json_to_images(query)
        
        for image in temp:
            images.append(image)
            
    return images

In [ ]:
#-- API Calling (limit: 50/hour) --
bird_queries = unsplash_term_queries('bird', 3)

In [ ]:
bird_supplement = queries_to_images(bird_queries)
        
len(bird_supplement)